In [2]:
import sys
!{sys.executable} -m pip install yfinance

     |████████████████████████████████| 5.4 MB 2.2 MB/s eta 0:00:01     |█████████████████████████▌      | 4.3 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 1.2 MB/s eta 0:00:01    |███▋                            | 1.7 MB 2.4 MB/s eta 0:00:06     |████████████▍                   | 5.7 MB 2.6 MB/s eta 0:00:04     |█████████████▏                  | 6.1 MB 2.6 MB/s eta 0:00:04
     |████████████████████████████████| 9.7 MB 361 kB/s eta 0:00:01     |████████████▋                   | 3.8 MB 1.3 MB/s eta 0:00:05     |█████████████████▋              | 5.3 MB 1.5 MB/s eta 0:00:03     |████████████████████▉           | 6.3 MB 1.5 MB/s eta 0:00:03
     |████████████████████████████████| 510 kB 1.1 MB/s eta 0:00:01
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=b47357754ef4cdbf7ddf723a8733cc0c7bb84ee8dd969aab01c83eef18999a9d
  Stored in directory: /home/jovyan/.cache/pip/wheels/b4/c3/39/9c01ae2b4726f37024bba5592bec868b47a2f

* Choose your favorite Stock
* Download its price series for the last 1 year
* Calculate daily returns
* Calculate the average of daily returns (multiply by 252 to make it annual)
* Calculate the daily variance and volatility


In [22]:
import yfinance as yf
# Get the data of the stock AAPL
data = yf.download('AAPL','2019-08-01','2020-08-01')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-08-01,53.474998,54.507500,51.685001,52.107498,51.311756,216071600
2019-08-02,51.382500,51.607498,50.407501,51.005001,50.226093,163448400
2019-08-05,49.497501,49.662498,48.145000,48.334999,47.596859,209572000
2019-08-06,49.077499,49.517502,48.509998,49.250000,48.497894,143299200
2019-08-07,48.852501,49.889999,48.455002,49.759998,49.000103,133457600
...,...,...,...,...,...,...
2020-07-27,93.709999,94.904999,93.480003,94.809998,94.476364,121214000
2020-07-28,94.367500,94.550003,93.247498,93.252502,92.924355,103625600
2020-07-29,93.750000,95.230003,93.712502,95.040001,94.705559,90329200


In [67]:
import pandas as pd

In [23]:
daily_returns = data['Close'].pct_change(1)
daily_returns

Date
2019-08-01         NaN
2019-08-02   -0.021158
2019-08-05   -0.052348
2019-08-06    0.018930
2019-08-07    0.010355
                ...   
2020-07-27    0.023700
2020-07-28   -0.016428
2020-07-29    0.019168
2020-07-30    0.012100
2020-07-31    0.104689
Name: Close, Length: 253, dtype: float64

In [57]:
annual_return=daily_returns.mean()*252
print("Annual return: %s" % annual_return)

Annual return: 0.8020678136889838


In [60]:
print("Variance: %s" % (daily_returns.var(ddof=0)*252) )

Variance: 0.17694214427680652


In [54]:
print("Volatility: %s" % (daily_returns.std()*(252**0.5)) )

Variance: 0.4214820198523133


In [55]:
print("sharpe ratio: %s" % (annual_return/(daily_returns.std()*(252**0.5))) )

sharpe ratio: 1.902970413708341


## Part 2

In [87]:
spy = yf.download('SPY','2019-08-01','2020-08-01').rename( columns={'Adj Close': 'SPY'} )[['SPY']]
aapl = yf.download('AAPL','2019-08-01','2020-08-01').rename( columns={'Adj Close': 'AAPL'} )[['AAPL']]
stocks = pd.concat([ spy, aapl], axis=1)
stocks

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,SPY,AAPL
Date,,
2019-08-01,286.694855,51.311756
2019-08-02,284.536194,50.226093
2019-08-05,275.979279,47.596859
2019-08-06,279.849335,48.497894
2019-08-07,280.014648,49.000103
...,...,...
2020-07-27,320.564850,94.476364
2020-07-28,318.531708,92.924355
2020-07-29,322.449249,94.705559


In [88]:
stocks['Ret SPY'] = stocks['SPY'].pct_change(1)
stocks['Ret AAPL'] = stocks['AAPL'].pct_change(1)

In [105]:
covariance = stocks.cov(ddof=0)['Ret SPY']['Ret AAPL']*252
print("Covariance: %s" % (covariance) )

Covariance: 0.1203530331309105


In [106]:
market_variance = stocks['Ret SPY'].var(ddof=0)*252
print("Market Variance: %s" % market_variance )

Market Variance: 0.10656329968698726


In [107]:
beta = covariance / market_variance
print("Beta: %s" % beta )

Beta: 1.1294041521276874


Beta is the rate of the slope of the fitted curve on 'Ret SPY' points too